# TP2 - Market Basket Analysis 
INF8111 - Fouille de données, Été 2023
### Membres de l'équipe
    - Marco Mudenge (2137307)
    - Matthieu Souttre (2239547)


## Date et directives de remise
Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

**Date limite: Juin 04, 23:55**.


## Market Basket Analysis

Le *Market Basket Analysis* (MBA) est une technique d'analyse de la fouille de données qui permet de découvrir les associations entre les produits ou leur regroupement. En explorant des motifs intéressants à partir d'une vaste collection de données, le MBA vise à comprendre / révéler les comportements d'achat des clients en se basant sur la théorie selon laquelle si vous avez acheté un certain ensemble de produits, vous êtes plus (ou moins) susceptible d'acheter un autre groupe de produits. En d'autres termes, le MBA permet aux détaillants d'identifier la relation entre les articles que les clients achètent, révélant des tendances d'articles souvent achetés ensemble.

Une approche largement utilisée pour explorer ces motifs consiste à construire *** des règles d'association *** telles que
- **si** acheté *ITEM_1* **alors** achètera *ITEM_2* avec **confiance** *X*.

Ces associations n'ont pas à être des règles individuelles. Ils peuvent impliquer de nombreux éléments. Par exemple, une personne dans un supermarché peut ajouter des œufs dans son panier, puis le MBA peut suggérer qu'elle achètera également du pain et/ou de la farine:

+ **si**  acheté *OEUFS* **alors** achètera [*PAIN* avec confiance *0,2*; *FARINE* avec confiance 0,05].

Cependant, si la personne décide maintenant d'ajouter de la farine à son panier, la nouvelle règle d'association pourrait être comme ci-dessous, suggérant des ingrédients pour faire un gâteau.

+ **si** acheté [*OEUFS, FARINE*] **alors** achètera [*SUCRE* avec confiance 0,45; LEVURE avec confiance 0,12; *PAIN* avec confiance *0,03*].

Il existe de nombreux scénarios réels où le MBA joue un rôle central dans l'analyse des données, comme les transactions de supermarché, les commandes en ligne ou l'historique des cartes de crédit. Les spécialistes du marketing peuvent utiliser ces règles d'association pour organiser les produits corrélés plus près les uns des autres sur les étagères des magasins ou faire des suggestions en ligne afin que les clients achètent plus d'articles. Un MBA peut généralement aider les détaillants à répondre aux questions les suivantes:

- Quels articles sont souvent achetés ensemble ?
- Étant donné un panier, quels articles suggérer ?
- Comment placer les articles ensemble sur les étagères ?

### Objectif

Votre objectif dans ce TP est de développer un algorithme MBA pour révéler les motifs en créant des règles d'association dans un ensemble de données volumineux avec plus de trois millions de transactions de supermarché. Cependant, la collecte de règles d'association dans les grands ensembles de données est un problème très intensif en calcul, ce qui rend presque impossible leur exécution sans système distribué. Par conséquent, pour exécuter votre algorithme, vous aurez accès à un cluster de *cloud computing* distribué avec des centaines de cœurs.

À cette fin, un algorithme **MapReduce** sera implémenté avec le framework [Apache Spark](http://spark.apache.org), un système informatique distribué rapide. En résumé, Spark est un framework open source conçu avec une méthodologie *scale-out* qui en fait un outil très puissant pour les programmeurs ou les développeurs d'applications pour effectuer un volume massif de calculs et de traitement de données dans des environnements distribués. Spark fournit des API de haut niveau qui facilitent la création d'applications parallèles sans avoir à se soucier de la façon dont votre code et vos données sont parallélisés / distribués par le cluster informatique. Spark fait tout pour vous.

La mise en œuvre suivra l'algorithme d'analyse du panier de marché présenté par Jongwook Woo et Yuhang Xu (2012). L'image **workflow.pdf** illustre le flux de travail de l'algorithme et doit être utilisée pour consultation tout au long de ce TP. Les cases bleues sont celles où vous devez implémenter une méthode pour effectuer une fonction de mappage ou de réduction, et les cases grises représentent leur sortie attendue. **Toutes ces opérations sont expliquées en détail dans les sections suivantes.**

## 1. Configuration de Spark

Spark fonctionne sur les systèmes Windows et UNIX (par exemple, Linux, Mac OS). Il est facile d'exécuter Spark localement sur une seule machine - tout ce dont vous avez besoin est d'avoir Java installé sur votre système PATH, ou la variable d'environnement JAVA_HOME pointant vers une installation Java. Il est obligatoire que le **JDK v8** soit installé sur votre système, car Spark ne prend actuellement en charge que cette version. Si ce n'est pas le cas, accédez à [la page Web de Java](https://www.oracle.com/technetwork/java/javase/downloads/jdk8-downloads-2133151.html) pour télécharger et installer une machine virtuelle Java. N'oubliez pas de définir la variable d'environnement JAVA_HOME pour utiliser JDK v8 si votre installation ne le fait pas automatiquement.

L'interface entre Python et Spark se fait via **PySpark**, qui peut être installé en exécutant `pip install pyspark` ou configuré en suivant la séquence ci-dessous:

1. D'abord, allez sur http://spark.apache.org/downloads
2. Sélectionnez la dernière version de Spark et le package pré-construit pour Apache Hadoop 2.7
3. Cliquez pour télécharger **spark-2.4.5-bin-hadoop2.7.tgz** et décompressez-le dans le dossier de votre choix.
4. Ensuite, exportez les variables suivantes pour lier PYSPARK (l'interface python de Spark) à votre distribution python dans votre fichier `~/.bash_profile`.

``
export SPARK_HOME=/chemin/ vers / spark-2.4.5-bin-hadoop2.7
export PYTHONPATH="$SPARK_HOME/python:$SPARK_HOME/python/lib/py4j-0.10.7-src.zip:$SPARK_HOME/python/lib/pyspark.zip:$ PYTHONPATH"
export PYSPARK_PYTHON=/chemin/vers/votre/python3
``

5. Exécutez `source ~./bash_profile` pour effectuer les modifications et redémarrer cette session de notebook jupyter.

#### Alternative pour utiliser Google Colab

Si vous prévoyez d'utiliser la plateforme Google Colab, exécutez la cellule de code suivante pour configurer Spark:

In [1]:
import os
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
!pip install pyspark
!pip install plotly
!pip install plotly jupyterlab
!pip install panda

#### Testez votre Spark
À l'aide du code suivant, vous pouvez tester si Spark est installé correctement.

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.sql("select 'spark' as hello ")
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



### 1.1 Exemple de comptage de produits 

Pour tester votre installation et commencer à vous familiariser avec Spark, nous suivrons un exemple qui compte combien de fois les produits d'un toy dataset ont été achetés.

Le principal point d'entrée pour commencer la programmation avec Spark est [l'API RDD](https://spark.apache.org/docs/latest/rdd-programming-guide.html), une excellente abstraction Spark pour travailler avec MapReduce. RDD est une collection d'éléments partitionnés sur les nœuds du cluster qui peuvent fonctionner en parallèle. En d'autres termes, RDD est la façon dont Spark maintient vos données prêtes à exécuter une fonction (par exemple, une fonction Map ou une fonction reduce) en parallèle. **Ne vous inquiétez pas si cela semble toujours déroutant, il sera clair une fois que vous commencerez à l'implémenter**. Cependant, cela fait partie de ce TP d'étudier / consulter [Spark python API](https://spark.apache.org/docs/latest/api/python/) et d'apprendre à l'utiliser. Certaines fonctions utiles offertes par l'API RDD sont:

1. **map**: return a new RDD by applying a function to each element of this RDD.
2. **flatMap**: return a new RDD by first applying a function to all elements of this RDD, and then flattening the results. **Should be used when each entry will yield more than one mapped element**
3. **reduce**: reduces the elements of this RDD using the specified commutative and associative binary operator.
4. **reduceByKey**: merge the values for each key using an associative and commutative reduce function
5. **groupByKey**: group the values for each key in the RDD into a single sequence
6. **collect**: return a list that contains all of the elements in this RDD. **Should not be used when working with a lot of data**
7. **sample**: return a sampled subset of this RDD
8. **count**: return the number of elements in this RDD.
9. **filter**: return a new RDD containing only the elements that satisfy a predicate.

In [3]:
from pyspark.sql import SparkSession

bucket_path = 'gs://my_bucket_inf8111_tp2_ms/'
jupyter_path = bucket_path+'notebooks/jupyter/'
instacart_path = bucket_path+'instacart/'

def map_to_product(row):
    """
    Map each transaction into a set of KEY-VALUE elements.
    The KEY is the word (product) itself and the VALUE is its number of apparitions.
    """
    products = row.transaction.split(';') # split products from the column transaction
    for p in products:
        yield (p, 1)

def reduce_product_by_key(value1, value2):
    "Reduce the mapped objects to unique words by merging (summing ) their values"
    return value1+value2

# Initializates a object of SparkSession class, main entry point to Spark's funcionalites
spark = SparkSession.builder.getOrCreate()
        
# Read a toy dataset
toy = spark.read.csv(instacart_path+'toy.csv', header=True)
print("Toy dataset")
toy.show()

# Obtain a RDD object to call a map function
toy_rdd = toy.rdd
print("Toy dataframe as a RDD object (list of Row objects):\n\t", toy_rdd.collect())

# Map function to identify all products
toy_rdd = toy_rdd.flatMap(map_to_product)
print("\nMapped products:\n\t", toy_rdd.collect())

# Reduce function to merge values of elements that share the same KEY
toy_rdd = toy_rdd.reduceByKey(reduce_product_by_key)
print("\nReduced (merged) products:\n\t", toy_rdd.collect())

print("\nVisualizing as a dataframe:")
toy_rdd.toDF(["product", "count_product"]).show()

Toy dataset


+--------+-----------+
|order_id|transaction|
+--------+-----------+
|       1|    a;b;c;f|
|       2|    d;b;a;e|
|       3|        c;b|
|       4|        b;c|
+--------+-----------+

Toy dataframe as a RDD object (list of Row objects):
	 [Row(order_id='1', transaction='a;b;c;f'), Row(order_id='2', transaction='d;b;a;e'), Row(order_id='3', transaction='c;b'), Row(order_id='4', transaction='b;c')]



Mapped products:
	 [('a', 1), ('b', 1), ('c', 1), ('f', 1), ('d', 1), ('b', 1), ('a', 1), ('e', 1), ('c', 1), ('b', 1), ('b', 1), ('c', 1)]

Reduced (merged) products:
	 [('a', 2), ('b', 4), ('c', 3), ('f', 1), ('d', 1), ('e', 1)]

Visualizing as a dataframe:
+-------+-------------+
|product|count_product|
+-------+-------------+
|      a|            2|
|      b|            4|
|      c|            3|
|      f|            1|
|      d|            1|
|      e|            1|
+-------+-------------+



### 1.2 Travailler avec Spark Dataframe

Dans l'exemple ci-dessus, nous avons brièvement utilisé une classe Dataframe de Spark, mais uniquement pour obtenir un objet RDD avec ``toy.rdd`` et pour aficher les données sous forme de tableau structuré avec le ``show ()`` une fonction. Cependant, [Dataframe](http://spark.apache.org/docs/latest/api/python/) est une partie cruciale de la version actuelle de Spark et est construit sur l'API RDD. Il s'agit d'une collection distribuée de lignes sous des colonnes nommées, identique à une table dans une base de données relationnelle. Le Dataframe de Spark fonctionne de la même manière que [Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). En fait, nous pouvons exporter (obtenir) une Dataframe Spark vers (à partir de) ​​une Dataframe pandas avec la fonction ``toPandas()``  (``spark.createDataFrame``).

Une fonctionnalité centrale du Dataframe est de bénéficier du [Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html#sql), un module qui permet des requêtes SQL sur des données structurées. Par exemple, le même « exemple de comptage de produits » aurait pu être implémenté comme une séquence d'opérations SQL sur les données:

In [4]:
import pyspark.sql.functions as f

# Creates a new column, products, with all products appering in each transaction
print('New column \'products\': exploding the transaction\'s products to a new row')
df_toy = toy.withColumn('products', f.explode(f.split(toy.transaction, ';')))
df_toy.show()

# Performs a select query and group rows by the product name, aggreagating by counting
print('Couting unique products:')
df_toy.select(df_toy.products)\
      .groupBy(df_toy.products)\
      .agg(f.count('products').alias('count_product'))\
      .sort('count_product', ascending=False)\
      .show()

New column 'products': exploding the transaction's products to a new row
+--------+-----------+--------+
|order_id|transaction|products|
+--------+-----------+--------+
|       1|    a;b;c;f|       a|
|       1|    a;b;c;f|       b|
|       1|    a;b;c;f|       c|
|       1|    a;b;c;f|       f|
|       2|    d;b;a;e|       d|
|       2|    d;b;a;e|       b|
|       2|    d;b;a;e|       a|
|       2|    d;b;a;e|       e|
|       3|        c;b|       c|
|       3|        c;b|       b|
|       4|        b;c|       b|
|       4|        b;c|       c|
+--------+-----------+--------+

Couting unique products:
+--------+-------------+
|products|count_product|
+--------+-------------+
|       b|            4|
|       c|            3|
|       a|            2|
|       e|            1|
|       d|            1|
|       f|            1|
+--------+-------------+



En outre, les mêmes opérations SQL effectuées ci-dessus auraient pu être effectuées avec une requête en langage SQL traditionnel comme indiqué ci-dessous:

In [5]:
# Creates a relational table TOY in the Spark session
df_toy.createOrReplaceTempView("TOY")

spark.sql("SELECT t.products, COUNT(t.products) AS product_count"
          " FROM TOY t"
          " GROUP BY t.products"
          " ORDER BY product_count DESC").show()

+--------+-------------+
|products|product_count|
+--------+-------------+
|       b|            4|
|       c|            3|
|       a|            2|
|       d|            1|
|       f|            1|
|       e|            1|
+--------+-------------+



Ces concepts SQL sont mentionnés ici car ils nous seront utiles lors du TP, principalement dans la section 3, pour manipuler les données du supermarché, qui sont structurées en tableaux. Ainsi, si vous n'êtes pas familier avec SQL, il est recommandé de suivre un [tutoriel](https://www.w3schools.com/sql/) pour comprendre les bases.

## 2. Algorithme MBA
Les sections suivantes expliquent comment développer chaque étape de l'algorithme MapReduce pour notre application de supermarché. La figure workflow.pdf illustre chaque étape de l'algorithme.

### 2.1 Map to Patterns (10 points)
Pour un sous-ensemble de transactions (c'est-à-dire les lignes de notre toy dataset), chaque transaction doit être **mappée** vers un ensemble de *motifs d'achat* trouvés dans la transaction. Formellement, ces motifs sont des sous-ensembles de produits qui représentent un groupe d'articles achetés ensemble. 

Pour le framework MapReduce, chaque motif doit être créé comme un élément *KEY-VALUE*, où la KEY peut prendre la forme d'un singleton, d'une paire ou d'un trio de produits présents dans la transaction. Plus précisément, pour chaque transaction, la fonction de mappage doit générer tous les sous-ensembles **UNIQUE** possibles de taille **UN, DEUX ou TROIS**. La VALEUR associée à chaque KEY est le nombre de fois que la KEY est apparue dans la transaction (si nous supposons qu'aucun produit n'apparaît plus d'une fois dans la transaction, cette valeur est toujours égale à un).

Maintenant, implémentez la fonction **map_to_patterns** qui reçoit une transaction (une ligne du dataset) et retourne les motifs trouvés dans la transaction. Les éléments mappés sont un tuple (KEY, VALUE), où KEY est également un tuple de noms de produits. Il est crucial de noter que, puisque chaque entrée (transaction) de la fonction MAP produira **plus** un élément KEY-VALUE, un *flatMap* doit être invoqué pour cette étape.

Pour le toy dataset, la sortie attendue est similaire à:


<pre style="align:center; border:1px solid black;font-size: 8pt; line-height: 1.1; height: auto; width: 20em; padding-left:1px">
<code>
+---------------+-----------+
|       patterns|occurrences|
+---------------+-----------+
|         ('a',)|          1|
|     ('a', 'b')|          1|
|('a', 'b', 'c')|          1|
|('a', 'b', 'f')|          1|
|     ('a', 'c')|          1|
|('a', 'c', 'f')|          1|
|     ('a', 'f')|          1|
|         ('b',)|          1|
|     ('b', 'c')|          1|
|('b', 'c', 'f')|          1|
|     ('b', 'f')|          1|
|         ('c',)|          1|
|     ('c', 'f')|          1|
|         ('f',)|          1|
|         ('a',)|          1|
|     ('a', 'b')|          1|
|('a', 'b', 'd')|          1|
|('a', 'b', 'e')|          1|
|     ('a', 'd')|          1|
|('a', 'd', 'e')|          1|
|     ('a', 'e')|          1|
|         ('b',)|          1|
|     ('b', 'd')|          1|
|('b', 'd', 'e')|          1|
|     ('b', 'e')|          1|
|         ('d',)|          1|
|     ('d', 'e')|          1|
|         ('e',)|          1|
|         ('b',)|          1|
|     ('b', 'c')|          1|
|         ('c',)|          1|
|         ('b',)|          1|
|     ('b', 'c')|          1|
|         ('c',)|          1|
+---------------+-----------+
</code>
</pre>

In [6]:
from itertools import combinations
import pandas as pd

def format_tuples(pattern):
    """
    Used for visualizition.
    Transforms tuples to a string since Dataframe does not support column of tuples with different sizes
    (a,b,c) -> '(a,b,c)'
    """
    return (str(pattern[0]), str(pattern[1]))

def map_to_patterns(row):
    """
    Map unique combinations of products for a single line.
    """
    products = sorted(row.transaction.split(';'))
    combinations_list = []
    for p in range(1, 4):
        order_combinations = list(combinations(products, p))
        combinations_list.extend(order_combinations)
        
    # Tri ici pour avoir le même ordre que l'exemple
    combinations_list = sorted(combinations_list)
    
    for c in combinations_list:
        yield (c, 1)

toy_rdd = toy.rdd
patterns_rdd = toy_rdd.flatMap(map_to_patterns)

# Output as dataframe
patterns_rdd.map(format_tuples).toDF(['patterns', 'occurrences']).show(50)

+---------------+-----------+
|       patterns|occurrences|
+---------------+-----------+
|         ('a',)|          1|
|     ('a', 'b')|          1|
|('a', 'b', 'c')|          1|
|('a', 'b', 'f')|          1|
|     ('a', 'c')|          1|
|('a', 'c', 'f')|          1|
|     ('a', 'f')|          1|
|         ('b',)|          1|
|     ('b', 'c')|          1|
|('b', 'c', 'f')|          1|
|     ('b', 'f')|          1|
|         ('c',)|          1|
|     ('c', 'f')|          1|
|         ('f',)|          1|
|         ('a',)|          1|
|     ('a', 'b')|          1|
|('a', 'b', 'd')|          1|
|('a', 'b', 'e')|          1|
|     ('a', 'd')|          1|
|('a', 'd', 'e')|          1|
|     ('a', 'e')|          1|
|         ('b',)|          1|
|     ('b', 'd')|          1|
|('b', 'd', 'e')|          1|
|     ('b', 'e')|          1|
|         ('d',)|          1|
|     ('d', 'e')|          1|
|         ('e',)|          1|
|         ('b',)|          1|
|     ('b', 'c')|          1|
|         

### 2.2  Reduce patterns  (2,5 points)
Une fois que différents processeurs ont traité les transactions, une fonction **reduce** doit être appelée pour combiner des KEYS identiques (le sous-ensemble de produits) et calculer le nombre total de ses occurrences dans le dataset. En d'autres termes, cette procédure de réduction doit additionner la *VALUE* de chaque KEY identique.

Créez ci-dessous une fonction **reduce_patterns** qui doit additionner la VALUE de chaque motif.
Pour le toy dataset, la sortie attendue est:
<pre style="align:center; border:1px solid black;font-size: 8pt; line-height: 1.1; height: auto; width: 28em; padding-left:5px">
<code>
+---------------+--------------------+
|       patterns|combined_occurrences|
+---------------+--------------------+
|         ('a',)|                   2|
|     ('a', 'b')|                   2|
|('a', 'b', 'c')|                   1|
|('a', 'b', 'f')|                   1|
|     ('a', 'c')|                   1|
|('a', 'c', 'f')|                   1|
|     ('a', 'f')|                   1|
|         ('b',)|                   4|
|     ('b', 'c')|                   3|
|('b', 'c', 'f')|                   1|
|     ('b', 'f')|                   1|
|         ('c',)|                   3|
|     ('c', 'f')|                   1|
|         ('f',)|                   1|
|('a', 'b', 'd')|                   1|
|('a', 'b', 'e')|                   1|
|     ('a', 'd')|                   1|
|('a', 'd', 'e')|                   1|
|     ('a', 'e')|                   1|
|     ('b', 'd')|                   1|
|('b', 'd', 'e')|                   1|
|     ('b', 'e')|                   1|
|         ('d',)|                   1|
|     ('d', 'e')|                   1|
|         ('e',)|                   1|
+---------------+--------------------+
</code>
</pre>


In [7]:
def reduce_patterns(value1, value2):
    """
    Reduce the mapped objects to unique words
    by merging (summing ) their values.
    As done in exemple 1.1
    """
    return value1+value2


combined_patterns_rdd = patterns_rdd.reduceByKey(reduce_product_by_key)

# Output as dataframe
combined_patterns_rdd.map(format_tuples).toDF(['patterns', 'combined_occurrences']).show(25)

+---------------+--------------------+
|       patterns|combined_occurrences|
+---------------+--------------------+
|         ('a',)|                   2|
|     ('a', 'b')|                   2|
|('a', 'b', 'c')|                   1|
|('a', 'b', 'f')|                   1|
|     ('a', 'c')|                   1|
|('a', 'c', 'f')|                   1|
|     ('a', 'f')|                   1|
|         ('b',)|                   4|
|     ('b', 'c')|                   3|
|('b', 'c', 'f')|                   1|
|     ('b', 'f')|                   1|
|         ('c',)|                   3|
|     ('c', 'f')|                   1|
|         ('f',)|                   1|
|('a', 'b', 'd')|                   1|
|('a', 'b', 'e')|                   1|
|     ('a', 'd')|                   1|
|('a', 'd', 'e')|                   1|
|     ('a', 'e')|                   1|
|     ('b', 'd')|                   1|
|('b', 'd', 'e')|                   1|
|     ('b', 'e')|                   1|
|         ('d',)|        

### 2.3 Map to subpatterns (15 points)
Ensuite, une autre fonction **map** doit être appliquée pour générer des sous-motifs. Encore une fois, les sous-motifs sont des éléments KEY-VALUE, où la KEY est également un sous-ensemble de produits. Cependant, la création de la KEY du sous-motif est une procédure différente. Cette fois, l'idée est de décomposer la liste des produits de chaque motif (KEY), de supprimer un produit à la fois et de produire la liste résultante en tant que nouvelle clé de sous-motif.

Par exemple, pour un modèle donné $P$ avec trois produits, $p_1, p_2$ et $p_3$, trois nouvelles clés de sous-motifs vont être créées: (i) supprimer $p_1$ et retourner ($p_2, p_3$) ; (ii) supprimer $p_2$ et retourner ($p_1, p_3$); et (iii) supprimer $p_3$ et retourner ($p_1, p_2$).

De plus, la structure VALUE du sous-motif sera également différente. Au lieu d'une seule valeur entière unique comme nous l'avons eu dans les motifs, cette fois un *tuple* devrait être créé pour le sous-motif VALUE. Ce tuple contient le produit qui a été retiré lors de la remise de la KEY et le nombre de fois que le motif est apparu. Par exemple ci-dessus, les valeurs doivent être ($p_1,v$), ($p_2,v$) et ($p_3,v $), respectivement, où $v$ est la VALEUR du motif.

L'idée derrière les sous-motif est de créer **des règles** telles que : lorsque les produits de KEY ont été achetés, l'article présent dans la VALEUR a également été acheté *v* fois. En outre, chaque motif doit également produire un sous-motif dans lequel la clé est la même liste de produits du motif, mais la valeur est un tuple avec un produit nul (None) et le nombre de fois que le motif est apparu. Cet élément sera utile pour garder une trace du nombre de fois où un tel motif a été trouvé et sera utilisé ultérieurement pour calculer la valeur de confiance lors de la génération des règles d'association.

Maintenant, implémentez la fonction **map_to_subpatterns** qui reçoit un motif et produit tous les sous-motif trouvés. Encore une fois, chaque entrée (motif) générera plus d'un élément KEY-VALUE, puis une fonction flatMap doit être appelée.

Pour le toy dataset, la sortie attendue est:

<pre style="align:center; border:1px solid black;font-size: 8pt; line-height: 1.1; height: auto; width: 20em; padding-left:5px">
<code>
+---------------+---------+
|    subpatterns|    rules|
+---------------+---------+
|         ('a',)|(None, 2)|
|     ('a', 'b')|(None, 2)|
|         ('b',)| ('a', 2)|
|         ('a',)| ('b', 2)|
|('a', 'b', 'c')|(None, 1)|
|     ('b', 'c')| ('a', 1)|
|     ('a', 'c')| ('b', 1)|
|     ('a', 'b')| ('c', 1)|
|('a', 'b', 'f')|(None, 1)|
|     ('b', 'f')| ('a', 1)|
|     ('a', 'f')| ('b', 1)|
|     ('a', 'b')| ('f', 1)|
|     ('a', 'c')|(None, 1)|
|         ('c',)| ('a', 1)|
|         ('a',)| ('c', 1)|
|('a', 'c', 'f')|(None, 1)|
|     ('c', 'f')| ('a', 1)|
|     ('a', 'f')| ('c', 1)|
|     ('a', 'c')| ('f', 1)|
|     ('a', 'f')|(None, 1)|
|         ('f',)| ('a', 1)|
|         ('a',)| ('f', 1)|
|         ('b',)|(None, 4)|
|     ('b', 'c')|(None, 3)|
|         ('c',)| ('b', 3)|
|         ('b',)| ('c', 3)|
|('b', 'c', 'f')|(None, 1)|
|     ('c', 'f')| ('b', 1)|
|     ('b', 'f')| ('c', 1)|
|     ('b', 'c')| ('f', 1)|
|     ('b', 'f')|(None, 1)|
|         ('f',)| ('b', 1)|
|         ('b',)| ('f', 1)|
|         ('c',)|(None, 3)|
|     ('c', 'f')|(None, 1)|
|         ('f',)| ('c', 1)|
|         ('c',)| ('f', 1)|
|         ('f',)|(None, 1)|
|('a', 'b', 'd')|(None, 1)|
|     ('b', 'd')| ('a', 1)|
|     ('a', 'd')| ('b', 1)|
|     ('a', 'b')| ('d', 1)|
|('a', 'b', 'e')|(None, 1)|
|     ('b', 'e')| ('a', 1)|
|     ('a', 'e')| ('b', 1)|
|     ('a', 'b')| ('e', 1)|
|     ('a', 'd')|(None, 1)|
|         ('d',)| ('a', 1)|
|         ('a',)| ('d', 1)|
|('a', 'd', 'e')|(None, 1)|
|     ('d', 'e')| ('a', 1)|
|     ('a', 'e')| ('d', 1)|
|     ('a', 'd')| ('e', 1)|
|     ('a', 'e')|(None, 1)|
|         ('e',)| ('a', 1)|
|         ('a',)| ('e', 1)|
|     ('b', 'd')|(None, 1)|
|         ('d',)| ('b', 1)|
|         ('b',)| ('d', 1)|
|('b', 'd', 'e')|(None, 1)|
|     ('d', 'e')| ('b', 1)|
|     ('b', 'e')| ('d', 1)|
|     ('b', 'd')| ('e', 1)|
|     ('b', 'e')|(None, 1)|
|         ('e',)| ('b', 1)|
|         ('b',)| ('e', 1)|
|         ('d',)|(None, 1)|
|     ('d', 'e')|(None, 1)|
|         ('e',)| ('d', 1)|
|         ('d',)| ('e', 1)|
|         ('e',)|(None, 1)|
+---------------+---------+
</code>
</pre>

In [8]:
from copy import deepcopy
def map_to_subpatterns(pattern):
    """
    Show the sub-patterns
    """
    yield (pattern[0], tuple((None, pattern[1])))
    if len(pattern[0]) > 1:
        for item in pattern[0]:
            yield(tuple(sub_pattern for sub_pattern in pattern[0] if sub_pattern != item), 
                  tuple((item, pattern[1])))

subpatterns_rdd = combined_patterns_rdd.flatMap(map_to_subpatterns)

# Output as dataframe
subpatterns_rdd.map(format_tuples).toDF(['subpatterns', 'rules']).show(100)

+---------------+---------+
|    subpatterns|    rules|
+---------------+---------+
|         ('a',)|(None, 2)|
|     ('a', 'b')|(None, 2)|
|         ('b',)| ('a', 2)|
|         ('a',)| ('b', 2)|
|('a', 'b', 'c')|(None, 1)|
|     ('b', 'c')| ('a', 1)|
|     ('a', 'c')| ('b', 1)|
|     ('a', 'b')| ('c', 1)|
|('a', 'b', 'f')|(None, 1)|
|     ('b', 'f')| ('a', 1)|
|     ('a', 'f')| ('b', 1)|
|     ('a', 'b')| ('f', 1)|
|     ('a', 'c')|(None, 1)|
|         ('c',)| ('a', 1)|
|         ('a',)| ('c', 1)|
|('a', 'c', 'f')|(None, 1)|
|     ('c', 'f')| ('a', 1)|
|     ('a', 'f')| ('c', 1)|
|     ('a', 'c')| ('f', 1)|
|     ('a', 'f')|(None, 1)|
|         ('f',)| ('a', 1)|
|         ('a',)| ('f', 1)|
|         ('b',)|(None, 4)|
|     ('b', 'c')|(None, 3)|
|         ('c',)| ('b', 3)|
|         ('b',)| ('c', 3)|
|('b', 'c', 'f')|(None, 1)|
|     ('c', 'f')| ('b', 1)|
|     ('b', 'f')| ('c', 1)|
|     ('b', 'c')| ('f', 1)|
|     ('b', 'f')|(None, 1)|
|         ('f',)| ('b', 1)|
|         ('b',)| ('

### 2.4 Reduce Subpatterns (2.5 points)

Encore une fois, une fonction **reduce** est nécessaire pour regrouper tous les sous-motif par leur KEY. L'objectif de cette procédure de réduction est de créer une liste de toutes les **règles** apparues dans KEY. Par conséquent, la sortie attendue résultant de cette fonction de réduction est également un élément KEY-VALUE, où la clé est la KEY du sous-motif et la valeur est un groupe contenant toutes les valeurs des sous-motif qui partagent la même clé.

Pour le toy dataset, la sortie attendue est:


<pre style="align:center; border:1px solid black;font-size: 8pt; line-height: 1.1; height: auto; width: 50em; padding-left:5px">
<code>
+---------------+-------------------------------------------------------------+
|subpatterns    |combined_rules                                               |
+---------------+-------------------------------------------------------------+
|('a',)         |[(None, 2), ('b', 2), ('c', 1), ('f', 1), ('d', 1), ('e', 1)]|
|('a', 'b')     |[(None, 2), ('c', 1), ('f', 1), ('d', 1), ('e', 1)]          |
|('b',)         |[('a', 2), (None, 4), ('c', 3), ('f', 1), ('d', 1), ('e', 1)]|
|('a', 'b', 'c')|[(None, 1)]                                                  |
|('b', 'c')     |[('a', 1), (None, 3), ('f', 1)]                              |
|('a', 'c')     |[('b', 1), (None, 1), ('f', 1)]                              |
|('a', 'b', 'f')|[(None, 1)]                                                  |
|('b', 'f')     |[('a', 1), ('c', 1), (None, 1)]                              |
|('a', 'f')     |[('b', 1), ('c', 1), (None, 1)]                              |
|('c',)         |[('a', 1), ('b', 3), (None, 3), ('f', 1)]                    |
|('a', 'c', 'f')|[(None, 1)]                                                  |
|('c', 'f')     |[('a', 1), ('b', 1), (None, 1)]                              |
|('f',)         |[('a', 1), ('b', 1), ('c', 1), (None, 1)]                    |
|('b', 'c', 'f')|[(None, 1)]                                                  |
|('a', 'b', 'd')|[(None, 1)]                                                  |
|('b', 'd')     |[('a', 1), (None, 1), ('e', 1)]                              |
|('a', 'd')     |[('b', 1), (None, 1), ('e', 1)]                              |
|('a', 'b', 'e')|[(None, 1)]                                                  |
|('b', 'e')     |[('a', 1), ('d', 1), (None, 1)]                              |
|('a', 'e')     |[('b', 1), ('d', 1), (None, 1)]                              |
+---------------+-------------------------------------------------------------+
</code>
</pre>


In [9]:
combined_rules = subpatterns_rdd.groupByKey().mapValues(list)

# Output as dataframe
combined_rules.map(format_tuples).toDF(['subpatterns', 'combined_rules']).show(truncate=False)

+---------------+-------------------------------------------------------------+
|subpatterns    |combined_rules                                               |
+---------------+-------------------------------------------------------------+
|('a',)         |[(None, 2), ('b', 2), ('c', 1), ('f', 1), ('d', 1), ('e', 1)]|
|('a', 'b')     |[(None, 2), ('c', 1), ('f', 1), ('d', 1), ('e', 1)]          |
|('b',)         |[('a', 2), (None, 4), ('c', 3), ('f', 1), ('d', 1), ('e', 1)]|
|('a', 'b', 'c')|[(None, 1)]                                                  |
|('b', 'c')     |[('a', 1), (None, 3), ('f', 1)]                              |
|('a', 'c')     |[('b', 1), (None, 1), ('f', 1)]                              |
|('a', 'b', 'f')|[(None, 1)]                                                  |
|('b', 'f')     |[('a', 1), ('c', 1), (None, 1)]                              |
|('a', 'f')     |[('b', 1), ('c', 1), (None, 1)]                              |
|('c',)         |[('a', 1), ('b', 3), (N

### 2.5. Map to Association Rules (15 points)

Enfin, la dernière étape de l'algorithme consiste à créer les règles d'association pour effectuer la MBA. Le but de cette fonction Map est de calculer le niveau **de confiance** de l'achat d'un produit, sachant qu'il y a déjà un ensemble de produits dans le panier. Ainsi, la KEY du sous-motif est l'ensemble des produits placés dans le panier et, pour chaque produit présent dans la liste des règles, c'est-à-dire dans la VALEUR, la confiance peut être calculée comme :

\begin{align*}
\frac{\text{nombre de fois où le produit a été acheté avec KEY}}{\text{nombre de fois où la KEY est apparue}}
\end{align*}

Pour l'exemple donné dans la figure "workflow", *le café* a été acheté 20 fois et, dans 17 d'entre eux, le *lait* a été acheté ensemble. Ensuite, le niveau de confiance pour acheter du *lait* sachant que *le café* est dans le panier est $\frac{17}{20}=0,85$, ce qui signifie que dans 85% des cas où le café a été acheté, le lait a aussi été acheté.

Implémentez la fonction **map_to_assoc_rules** qui calcule le niveau de confiance pour chaque sous-motif.

Pour le toy dataset, la sortie attendue est:
<pre style="align:center; border:1px solid black;font-size: 8pt; line-height: 1.1; height: auto; width: 57em; padding-left:5px">
<code>
+---------------+------------------------------------------------------------------+
|patterns       |association_rules                                                 |
+---------------+------------------------------------------------------------------+
|('a',)         |[('b', 1.0), ('c', 0.5), ('f', 0.5), ('d', 0.5), ('e', 0.5)]      |
|('a', 'b')     |[('c', 0.5), ('f', 0.5), ('d', 0.5), ('e', 0.5)]                  |
|('b',)         |[('a', 0.5), ('c', 0.75), ('f', 0.25), ('d', 0.25), ('e', 0.25)]  |
|('a', 'b', 'c')|[]                                                                |
|('b', 'c')     |[('a', 0.3333333333333333), ('f', 0.3333333333333333)]            |
|('a', 'c')     |[('b', 1.0), ('f', 1.0)]                                          |
|('a', 'b', 'f')|[]                                                                |
|('b', 'f')     |[('a', 1.0), ('c', 1.0)]                                          |
|('a', 'f')     |[('b', 1.0), ('c', 1.0)]                                          |
|('c',)         |[('a', 0.3333333333333333), ('b', 1.0), ('f', 0.3333333333333333)]|
|('a', 'c', 'f')|[]                                                                |
|('c', 'f')     |[('a', 1.0), ('b', 1.0)]                                          |
|('f',)         |[('a', 1.0), ('b', 1.0), ('c', 1.0)]                              |
|('b', 'c', 'f')|[]                                                                |
|('a', 'b', 'd')|[]                                                                |
|('b', 'd')     |[('a', 1.0), ('e', 1.0)]                                          |
|('a', 'd')     |[('b', 1.0), ('e', 1.0)]                                          |
|('a', 'b', 'e')|[]                                                                |
|('b', 'e')     |[('a', 1.0), ('d', 1.0)]                                          |
|('a', 'e')     |[('b', 1.0), ('d', 1.0)]                                          |
+---------------+------------------------------------------------------------------+
</code>
</pre>

In [10]:
def map_to_assoc_rules(rule):
    """
    Map the association rules
    """
    association_rules = []
    
    for i in rule[1]:
        if i[0] is None:
            key_occurences = i[1]
    
    for i in rule[1]:
        if i[0] is not None:
            association_rules.extend(tuple((i[0], i[1]/key_occurences)))
            
    yield(rule[0],association_rules)

assoc_rules = combined_rules.flatMap(map_to_assoc_rules)

# Output as dataframe
assoc_rules.map(format_tuples).toDF(['patterns', 'association_rules']).show(truncate=False)

+---------------+------------------------------------------------------------+
|patterns       |association_rules                                           |
+---------------+------------------------------------------------------------+
|('a',)         |['b', 1.0, 'c', 0.5, 'f', 0.5, 'd', 0.5, 'e', 0.5]          |
|('a', 'b')     |['c', 0.5, 'f', 0.5, 'd', 0.5, 'e', 0.5]                    |
|('b',)         |['a', 0.5, 'c', 0.75, 'f', 0.25, 'd', 0.25, 'e', 0.25]      |
|('a', 'b', 'c')|[]                                                          |
|('b', 'c')     |['a', 0.3333333333333333, 'f', 0.3333333333333333]          |
|('a', 'c')     |['b', 1.0, 'f', 1.0]                                        |
|('a', 'b', 'f')|[]                                                          |
|('b', 'f')     |['a', 1.0, 'c', 1.0]                                        |
|('a', 'f')     |['b', 1.0, 'c', 1.0]                                        |
|('c',)         |['a', 0.3333333333333333, 'b', 1.0,

## 3. Instacart dataset

Avec votre algorithme MBA prêt à être utilisé, il est maintenant temps de travailler sur l'ensemble de données réel. Pour cette partie du TP, téléchargez le dataset [instacart](https://drive.google.com/file/d/1pXjqPz1RbL40yCGWnTCbmW_ZXrjlfJi4/view?usp=sharing) et lisez sa [description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) pour comprendre la structure de l'ensemble de données.

Avant d'appliquer l'algorithme développé sur l'ensemble de données instacart, vous devez d'abord filtrer les transactions pour qu'elles soient au même format défini par votre algorithme (une transaction par ligne). Pour manipuler les données, nous pouvons utiliser le bloc de données de Spark et le module SQL présenté dans la section 1.

La cellule de code suivante utilise le module Spark SQL pour lire les commandes de ``order_products__train.csv`` et les informations détaillées de ``orders.csv`` et ``products.csv`` pour construire une dataframe qui contient un liste de tous les produits jamais achetés par chaque utilisateur.

In [11]:
df_order_prod = spark.read.csv(bucket_path + 'instacart/order_products__train.csv', header=True, sep=',', inferSchema=True)
print('order_products__train.csv')
df_order_prod.show(5)

df_orders = spark.read.csv(bucket_path + 'instacart/orders.csv', header=True, sep=',', inferSchema=True)
print('orders.csv')
df_orders.show(5)

df_products = spark.read.csv(bucket_path + 'instacart/products.csv', header=True, sep=',', inferSchema=True)
print('products.csv')
df_products.show(5)

df_departments = spark.read.csv(bucket_path + 'instacart/departments.csv', header=True, sep=',', inferSchema=True)
print('departments.csv')
df_departments.show(5)

"""
List of products ever purchased by each user
"""
# USING SQL
df_order_prod.createOrReplaceTempView("order_prod") # creates table 'order_prod'
df_orders.createOrReplaceTempView("orders") # creates table 'orders'
df_products.createOrReplaceTempView("products") # creates table 'products'
df_departments.createOrReplaceTempView("departments") # creates table 'departments'
spark.sql('SELECT o.user_id, COLLECT_LIST(p.product_name) AS products' 
               ' FROM orders o '
               ' INNER JOIN order_prod op ON op.order_id = o.order_id'
               ' INNER JOIN products p    ON op.product_id = p.product_id'
               ' GROUP BY user_id ORDER BY o.user_id').show(5, truncate=80)


# USING DATAFRAME OPERATIONS
# df_orders.join(df_order_prod, df_order_prod.order_id == df_orders.order_id, 'inner')\
# .join(df_products, df_products.product_id == df_order_prod.product_id, 'inner')\
# .groupBy(df_orders.user_id).agg(f.collect_list(df_products.product_name).alias('products'))\
# .orderBy(df_orders.user_id).show(5, truncate=80)

order_products__train.csv
+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
+--------+----------+-----------------+---------+
only showing top 5 rows



orders.csv
+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|                8|                  null|
| 2398795|      1|   prior|           2|        3|                7|                  15.0|
|  473747|      1|   prior|           3|        3|               12|                  21.0|
| 2254736|      1|   prior|           4|        4|                7|                  29.0|
|  431534|      1|   prior|           5|        4|               15|                  28.0|
+--------+-------+--------+------------+---------+-----------------+----------------------+
only showing top 5 rows

products.csv
+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+--------

+-------+--------------------------------------------------------------------------------+
|user_id|                                                                        products|
+-------+--------------------------------------------------------------------------------+
|      1|[Zero Calorie Cola, Organic Half & Half, Organic Whole Milk, Aged White Chedd...|
|      2|[Organic Hearty Split Pea & Uncured Ham Soup, Organic Cashew Carrot Ginger So...|
|      5|[Organic Baby Arugula, Organic Grape Tomatoes, Tamari Gluten Free Soy Sauce, ...|
|      7|[Honeycrisp Apple, Organic Dark Brown Sugar, Vanilla Coffee Concentrate, Lact...|
|      8|[Organic Green Onions, Solid White-No Salt Added Albacore Tuna, Organic Whole...|
+-------+--------------------------------------------------------------------------------+
only showing top 5 rows



### 3.1 Perspectives commerciales (20 points) 

Maintenant, vous êtes le *data scientist*. En ne considérant que les commandes de ``order_products__train.csv``, l'utilisation du module Spark SQL, performant avec SQL ou dataframe, pour répondre aux questions suivantes:

1. Quels sont les 10 produits les plus susceptibles d'être commandé de nouveau? Ne considérez que les produits achetés au moins 40 fois pour cette tâche.
2. Quels sont les 3 produits les plus achetés dans chaque département?
4. Quelle est la taille moyenne du panier pour chaque jour de la semaine?
    - utilisez un barplot pour visualiser vos résultats

**La sortie de ces questions doit contenir le NOM des produits, pas leur ID.**

In [12]:
%%time
Q1_sql = """
SELECT
  products.product_name,
  order_count,
  reordered,
  reorder_percentage
FROM
  (
    SELECT
      order_prod.product_id,
      COUNT(order_prod.product_id) AS order_count,
      COUNT(order_prod.reordered) AS reordered,
      SUM(order_prod.reordered) / COUNT(order_prod.product_id) AS reorder_percentage
    FROM
      order_prod
    GROUP BY
      product_id
  ) AS Table1
  LEFT JOIN products ON Table1.product_id = products.product_id
WHERE
  order_count >= 40
ORDER BY
  reorder_percentage DESC
"""

Q1_df = spark.sql(Q1_sql)
Q1_df.show(10)


+--------------------+-----------+---------+------------------+
|        product_name|order_count|reordered|reorder_percentage|
+--------------------+-----------+---------+------------------+
|2% Lactose Free Milk|         92|       92|0.9347826086956522|
|Organic Low Fat Milk|        368|      368|0.9130434782608695|
|100% Florida Oran...|         59|       59|0.8983050847457628|
|Original Sparklin...|         45|       45|0.8888888888888888|
|Organic Spelt Tor...|         81|       81|0.8888888888888888|
|              Banana|      18726|    18726|0.8841717398269785|
|  Petit Suisse Fruit|        120|      120|0.8833333333333333|
|Organic Lowfat 1%...|        483|      483|0.8819875776397516|
|Organic Lactose F...|        269|      269|0.8810408921933085|
|      1% Lowfat Milk|        461|      461|0.8785249457700651|
+--------------------+-----------+---------+------------------+
only showing top 10 rows

CPU times: user 3.58 ms, sys: 654 µs, total: 4.23 ms
Wall time: 1.13 s


In [13]:
# import plotly.express as px
# import plotly.graph_objects as go
# Q1_df_pd = Q1_df.toPandas()
# fig = px.bar(Q1_df_pd[1:10],
#              x="product_name",
#              y="reorder_percentage")
# fig.update_layout(xaxis_title='Products', yaxis_title='Reorder percentage')
# fig.show()

In [14]:
%%time
Q2_sql = """
SELECT *
FROM(
SELECT
  Table2.product_name,
  Table2.department_id,
  Table3.department,
  order_count,
  RANK() OVER (PARTITION BY Table2.department_id ORDER BY order_count DESC) AS ranking
FROM
  (
    SELECT
      order_prod.product_id,
      COUNT(order_prod.product_id) AS order_count
    FROM
      order_prod
    GROUP BY
      product_id
  ) AS Table1
  LEFT JOIN products Table2
      ON Table1.product_id = Table2.product_id
  LEFT JOIN departments Table3 
      ON Table2.department_id = Table3.department_id
ORDER  BY department_id ASC, order_count DESC)
Where ranking < 4
"""

Q2_df = spark.sql(Q2_sql)
Q2_df.show()

+--------------------+-------------+-------------+-----------+-------+
|        product_name|department_id|   department|order_count|ranking|
+--------------------+-------------+-------------+-----------+-------+
|         Blueberries|            1|       frozen|       2323|      1|
|Organic Broccoli ...|            1|       frozen|       1361|      2|
|Organic Whole Str...|            1|       frozen|       1213|      3|
|         Dried Mango|           10|         bulk|        446|      1|
| Organic Rolled Oats|           10|         bulk|        259|      2|
|Organic Black Mis...|           10|         bulk|        125|      3|
|  Lavender Hand Soap|           11|personal care|        258|      1|
|        Cotton Swabs|           11|personal care|        258|      1|
|Lemon Verbena Han...|           11|personal care|        191|      3|
|Boneless Skinless...|           12| meat seafood|       2088|      1|
|Ground Turkey Breast|           12| meat seafood|        958|      2|
|Bonel

In [15]:
# # Bar plot viz
# Q2_df_pd = Q2_df.toPandas()
# fig = px.histogram(Q2_df_pd,
#              x="department",
#              y="order_count",
#              barmode='group',
#             color='ranking',
#             hover_name="product_name")
# fig.update_layout(xaxis_title='Departments', yaxis_title='Product ordered')
# fig.show()

In [16]:
%%time
Q3_sql = """
SELECT
  order_dow,
  Avg(items_ordered) AS Daily_Average_Basket
FROM
  (
    SELECT
      order_prod.order_id,
      COUNT(order_prod.order_id) AS items_ordered
    FROM
      order_prod
    GROUP BY
      order_id
  ) AS Table1
  LEFT JOIN orders ON Table1.order_id = orders.order_id
GROUP BY order_dow
ORDER BY order_dow
"""

Q3_df = spark.sql(Q3_sql)
Q3_df.show()

+---------+--------------------+
|order_dow|Daily_Average_Basket|
+---------+--------------------+
|        0|  11.797778991443655|
|        1|  10.470618137454249|
|        2|    9.96103976673491|
|        3|    9.84133358832154|
|        4|   9.742527727301209|
|        5|  10.163736642537057|
|        6|  10.966562615734617|
+---------+--------------------+

CPU times: user 2.61 ms, sys: 3.13 ms, total: 5.74 ms
Wall time: 3.12 s


In [17]:
# # Bar plot viz

# import plotly.express as px
# Q3_df_pd = Q3_df.toPandas()
# fig = px.bar(Q3_df_pd,
#              x="order_dow",
#              y="Daily_Average_Basket")
# fig.update_layout(xaxis_title='Day of the week', yaxis_title='Daily average basket')
# fig.update_layout(xaxis=dict(tickmode='array', tickvals=[0,1,2,3,4,5,6], ticktext=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']))
# fig.show()

### 3.2 MBA pour le training set (15 points)

En utilisant les commandes du ``order_products__train.csv``, créez un bloc de données où chaque ligne contient la colonne ``transaction`` avec la liste des produits achetés, de manière similaire à le toy dataset. Ensuite, exécutez l'algorithme MBA pour cet ensemble de transactions.

- Vous devez signaler le temps passé pour effectuer cette tâche.
- La sortie doit contenir le nom des produits.

In [18]:
%%time
from pyspark.sql.functions import col, concat_ws

# The name of the order's products column 'transaction' to re-use the MAP of section 1
MBD_df_query = """
SELECT
  order_prod.order_id,
  concat_ws(";", collect_set(products.product_name)) AS transaction
FROM
  order_prod
  LEFT JOIN products ON products.product_id = order_prod.product_id
GROUP BY
  order_prod.order_id
"""

MBA_df = spark.sql(MBD_df_query)
MBA_df.show()


+--------+--------------------+
|order_id|         transaction|
+--------+--------------------+
|    1139|Cinnamon Rolls wi...|
|    1143|Water;Natural Pre...|
|    1342|Raw Shrimp;Seedle...|
|    1468|Pomegranate Seeds...|
|    1591|Cracked Wheat;Str...|
|    1721|Organic Thompson ...|
|    2711|Granny Smith Appl...|
|    2888|Shredded Sharp Ch...|
|    3179|Caramel Almond an...|
|    4092|Organic Baby Spin...|
|    4383|Organic Orange So...|
|    4519|Beet Apple Carrot...|
|    4935|               Vodka|
|    5117|Major Dickason's ...|
|    5433|Dairy Free Unswee...|
|    6357|Globe Eggplant;Pa...|
|    6552|Plain Almond Milk...|
|    6773|Unrefined Virgin ...|
|    7168|Dry Rubbed Thick ...|
|    7865|Taboule Salad;Org...|
+--------+--------------------+
only showing top 20 rows

CPU times: user 3.58 ms, sys: 671 µs, total: 4.25 ms
Wall time: 1.41 s


In [19]:
%%time
# MAP & REDUCE TO PATTERNS

MBA_rdd = MBA_df.rdd
MBA_patterns_rdd = MBA_rdd.flatMap(map_to_patterns)

# Output as dataframe
MBA_patterns_rdd.map(format_tuples).toDF(['patterns', 'occurrences']).show(10)

MBA_combined_patterns_rdd = MBA_patterns_rdd.reduceByKey(reduce_product_by_key)

# Output as dataframe
MBA_combined_patterns_rdd.map(format_tuples).toDF(['patterns', 'combined_occurrences']).show(10)

+--------------------+-----------+
|            patterns|occurrences|
+--------------------+-----------+
|         ('Banana',)|          1|
|('Banana', 'Butte...|          1|
|('Banana', 'Butte...|          1|
|('Banana', 'Butte...|          1|
|('Banana', 'Butte...|          1|
|('Banana', 'Butte...|          1|
|('Banana', 'Butte...|          1|
|('Banana', 'Butte...|          1|
|('Banana', 'Butte...|          1|
|('Banana', 'Cinna...|          1|
+--------------------+-----------+
only showing top 10 rows



+--------------------+--------------------+
|            patterns|combined_occurrences|
+--------------------+--------------------+
|('Organic Banana'...|                   1|
|('Crispy Wheat, C...|                   1|
|('Honestly Health...|                   1|
|('2% Shredded Sha...|                   1|
|('Gluten Free Ric...|                   1|
|('Frozen Organic ...|                   1|
|('Limes', 'Organi...|                   4|
|('Organic Garlic'...|                   1|
|('30% Less Sodium...|                   1|
|('Lemon Lime Soda...|                   2|
+--------------------+--------------------+
only showing top 10 rows

CPU times: user 60.8 ms, sys: 31.1 ms, total: 92 ms
Wall time: 4min 8s


In [20]:
%%time

# MAP & REDUCE TO SUBPATTERNS

MBA_subpatterns_rdd = MBA_combined_patterns_rdd.flatMap(map_to_subpatterns)

# Output as dataframe
MBA_subpatterns_rdd.map(format_tuples).toDF(['subpatterns', 'rules']).show(10)

MBA_combined_rules = MBA_subpatterns_rdd.groupByKey().mapValues(list)

# Output as dataframe
MBA_combined_rules.map(format_tuples).toDF(['subpatterns', 'combined_rules']).show(10)

+--------------------+--------------------+
|         subpatterns|               rules|
+--------------------+--------------------+
|('Bag of Organic ...|          (None, 29)|
|('Organic Black B...|('Bag of Organic ...|
|('Bag of Organic ...|('Organic Black B...|
|('Bag of Organic ...|('Organic Diced T...|
|('Balsamic Vinaig...|           (None, 1)|
|('Shredded Mild C...|('Balsamic Vinaig...|
|('Balsamic Vinaig...|('Shredded Mild C...|
|('Balsamic Vinaig...|('Vegetable Chick...|
|('Carrot Squash &...|           (None, 1)|
|('Dark Chocolate ...|('Carrot Squash &...|
+--------------------+--------------------+
only showing top 10 rows



+--------------------+--------------------+
|         subpatterns|      combined_rules|
+--------------------+--------------------+
|('Blackberry Pres...|[('Organic Strawb...|
|('Non Fat Black C...|[('Organic Hemp R...|
|('Coconut Blended...|[('Organic Dried ...|
|('Organic Corn St...|[('Organic Lightl...|
|('Green Peas', 'O...|[('Whole Milk Pla...|
|('Real Mayonnaise',)|[('Cheese Pizza R...|
|('Chopped Organic...|[('Super Puffs Sp...|
|('Diced Tomatoes'...|[('Greek Whole Mi...|
|('Honey Nut Cheer...|[('Almond Non-Dai...|
|('Almost Naked Pe...|[('Asparagus', 1)...|
+--------------------+--------------------+
only showing top 10 rows

CPU times: user 152 ms, sys: 22.9 ms, total: 175 ms
Wall time: 12min 20s


In [21]:
%%time

MBA_assoc_rules = MBA_combined_rules.flatMap(map_to_assoc_rules)

# Output as dataframe
MBA_assoc_rules.map(format_tuples).toDF(['patterns', 'association_rules']).show(10)

+--------------------+--------------------+
|            patterns|   association_rules|
+--------------------+--------------------+
|('100% Recycled P...|['Premium Fish Sa...|
|('Organic Dill', ...|['Frozen Organic ...|
|('Ataulfo Mango',...|['Sliced Black Ol...|
|('Original Whole ...|['Dijon Mustard',...|
|('Danimals Strawb...|['Bag of Organic ...|
|('Organic Whole W...|['Organic Iceberg...|
|('Craisins Dried ...|['Medium Roast Wh...|
|('ALMONDBREEZE UN...|['Unsweetened Van...|
|('Chocolate Lowfa...|['Uncured Beef Ho...|
|('70% Dark Chocol...|['Cold-Pressed Or...|
+--------------------+--------------------+
only showing top 10 rows

CPU times: user 36.7 ms, sys: 9.03 ms, total: 45.7 ms
Wall time: 3min 15s


# 3.3 MBA pour le dataset complet (20 points)

Comme vous l'avez probablement remarqué, même pour un ensemble de données moins volumineux (le training dataset ne contient que 131 000 commandes), l'algorithme MBA est coûteux en calcul. Pour cette raison, cette fois, nous allons répéter le processus, mais en utilisant maintenant Google Cloud Platform (GCP) pour créer un grand cluster. Toutes les instructions pour créer un cluster avec spark et comment soumettre un travail seront expliquées dans le laboratoire. Dans tous les cas, vous devez lire les instructions données dans le ``Instruction_GCP.pdf``.

Cette fois, nous travaillerons avec le fichier ``order_products__prior.csv``, qui contient plus de 3M commandes.

**PRODUCTION ATTENDUE**

Après avoir exécuté le MBA pour la plus grande collection de commandes, sélectionnez au hasard UN produit acheté dans ``order_products__prior`` et affichez les règles d'association (nom du produit et valeur d'association) de ce produit, c'est-à-dire lorsque le produit est seul dans le panier. La sortie doit être formatée dans un tableau, où chaque ligne contenant les informations d'un produit associé. 

- Affichez l'ID et le nom du produit sélectionné au hasard.
- Signaler le temps d'exécution.

**Remarque importante : joignez des captures d'écran de votre sortie et de votre configuration de cluster.** 

In [22]:
%%time
df_order_prior = spark.read.csv(bucket_path + 'instacart/order_products__prior.csv', header=True, sep=',', inferSchema=True)
print('order_products__prior.csv')
df_order_prior.show(5)

df_order_prior.createOrReplaceTempView("order_prior") # creates table 'order_prior'

order_products__prior.csv
+---+--------+----------+-----------------+---------+
|_c0|order_id|product_id|add_to_cart_order|reordered|
+---+--------+----------+-----------------+---------+
|  0|       2|     33120|                1|        1|
|  1|       2|     28985|                2|        1|
|  2|       2|      9327|                3|        0|
|  3|       2|     45918|                4|        1|
|  4|       2|     30035|                5|        0|
+---+--------+----------+-----------------+---------+
only showing top 5 rows

CPU times: user 9.47 ms, sys: 207 µs, total: 9.67 ms
Wall time: 2.56 s


In [23]:
%%time
# The name of the order's products column 'transaction' to re-use the MAP of section 1
FULL_MBD_df_query = """
SELECT
  order_prior.order_id,
  concat_ws(";", collect_set(products.product_name)) AS transaction
FROM
  order_prior
  LEFT JOIN products ON products.product_id = order_prior.product_id
GROUP BY
  order_prior.order_id
"""

FULL_MBA_df = spark.sql(FULL_MBD_df_query)
FULL_MBA_df.show()

+--------+--------------------+
|order_id|         transaction|
+--------+--------------------+
|      26|Organic Baby Spin...|
|      27|Firm Tofu;Parchme...|
|      28|Original Semisoft...|
|      31|Organic Cilantro;...|
|      44|French Onion Dip;...|
|      53|Organic Original ...|
|      65|Soy Sauce;Organic...|
|      76|Grape Tomatoes;Se...|
|      78|Roasted Pine Nut ...|
|      81|Sliced Baby Bella...|
|      85|Miniatures;Green ...|
|     101|Organic Baby Arug...|
|     103|House Blend Deep ...|
|     108|Thin Sliced Ameri...|
|     115|Marinara Sauce;Wi...|
|     126|Vanilla Whey Prot...|
|     133|Organic Garnet Sw...|
|     148|Organic Baby Spin...|
|     155|Premium Pure Cane...|
|     159|Organic Baby Arug...|
+--------+--------------------+
only showing top 20 rows

CPU times: user 8.09 ms, sys: 0 ns, total: 8.09 ms
Wall time: 3.15 s


In [24]:
%%time
# Rouler l'analyses MBA au complet avant sans les affichage

FULL_MBA_rdd = FULL_MBA_df.rdd

# Map & reduce patterns
FULL_MBA_patterns_rdd = FULL_MBA_rdd.flatMap(map_to_patterns)
FULL_MBA_combined_patterns_rdd = FULL_MBA_patterns_rdd.reduceByKey(reduce_product_by_key)

# Map & reduce to subpatterns
FULL_MBA_subpatterns_rdd = FULL_MBA_combined_patterns_rdd.flatMap(map_to_subpatterns)
FULL_MBA_combined_rules = FULL_MBA_subpatterns_rdd.groupByKey().mapValues(list)

# Combine association rules
FULL_MBA_assoc_rules = FULL_MBA_combined_rules.flatMap(map_to_assoc_rules)

CPU times: user 8.37 ms, sys: 8.87 ms, total: 17.2 ms
Wall time: 2.95 s


In [25]:
%%time
# Show patterns
FULL_MBA_patterns_rdd.map(format_tuples).toDF(['patterns', 'occurrences']).show(10)
FULL_MBA_combined_patterns_rdd.map(format_tuples).toDF(['patterns', 'combined_occurrences']).show(10)

# Show subpatterns
FULL_MBA_subpatterns_rdd.map(format_tuples).toDF(['subpatterns', 'rules']).show(10)
FULL_MBA_combined_rules.map(format_tuples).toDF(['subpatterns', 'combined_rules']).show(10)

# Show association rules
FULL_MBA_assoc_rules.map(format_tuples).toDF(['patterns', 'association_rules']).show(10)

## With one single product

In [56]:
from random import randint 

ord_df = spark.sql(
"""
SELECT order_prod.order_id, products.product_id, concat_ws(";", collect_set(products.product_name)) AS transaction 
  FROM order_prod
  INNER JOIN products ON products.product_id = order_prod.product_id
  GROUP BY order_prod.order_id, products.product_id 
  ORDER BY Rand() LIMIT 1
"""
)

ord_df.show(5, truncate=100)

+--------+----------+------------+
|order_id|product_id| transaction|
+--------+----------+------------+
| 3368469|      8069|Rice Vinegar|
+--------+----------+------------+



In [57]:
%%time
product_name = ord_df.select('transaction').collect()[0].transaction
print(product_name)

Rice Vinegar
CPU times: user 9.13 ms, sys: 1.44 ms, total: 10.6 ms
Wall time: 2.18 s


In [ ]:
%%time
res = FULL_MBA_assoc_rules.filter(lambda x: len(x[0]) == 1 and x[0][0] == product_name).map(format_tuples).toDF(['patterns', 'association_rules'])
res.show(truncate=300)

## Screenshots 

### Configuration du cluster

![cluster_config_1](screenshots/cluster_config_1.png)

![cluster_config_2](screenshots/cluster_config_2.png)

![cluster_config_3](screenshots/cluster_config_3.png)

### Sorties

![output_1](screenshots/output1.png)

![output_2](screenshots/output2.png)

![output_3](screenshots/output3.png)

![output_4](screenshots/output4.png)

![output_5](screenshots/output5.png)

### Produit seul

![output_single_prod_1](screenshots/output_single_prod_1.png)

![output_single_prod_2](screenshots/output_single_prod_2.png)